In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - orchestration
@forusone (shins777@gmail.com)

This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [ ]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    "langgraph" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [ ]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"

### Initialize Vertex AI

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

### Helper method to get session history

In [ ]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [ ]:
# Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
  """
     지정된 날짜의 두 통화 간 환율을 검색합니다.
  """

  import requests

  response = requests.get(
      f"https://api.frankfurter.app/{currency_date}",
      params={"from": currency_from, "to": currency_to},
  )
  return response.json()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

custom_prompt_template = {
    "user_input": lambda x: x["input"],
    "history": lambda x: x["history"],
    "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
} | ChatPromptTemplate.from_messages([
    ("placeholder", "{history}"),
    ("user", "{user_input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    planner = ChatPromptTemplate.from_template(
        "Generate an argument about: {input}"
    ) | model | output_parser

    pros = ChatPromptTemplate.from_template(
        "List the positive aspects of {input}"
    ) | model | output_parser

    cons = ChatPromptTemplate.from_template(
        "List the negative aspects of {input}"
    ) | model | output_parser

    summary = ChatPromptTemplate.from_template(
        "Input:{input}\nGenerate a final response given the critique",
    ) | model | output_parser

    builder = MessageGraph()
    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")
    return builder.compile()

agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    runnable_builder=langgraph_builder,
)


In [21]:
from time import perf_counter

t1_start = perf_counter()
response = agent.query(input={"role": "user", "content": "scrum methodology"})
t1_end = perf_counter()

display(Markdown(f"Elapse time : {t1_end-t1_start}"))

for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('======')

Elapse time : 6.986696317999986

scrum methodology

## The Agile Dilemma: Scrum's Strengths and Weaknesses

Scrum methodology, a framework for managing complex projects, has gained immense popularity in the tech industry and beyond. While lauded for its adaptability and collaborative approach, its effectiveness remains a subject of debate.

**Arguments for Scrum:**

* **Increased Flexibility:** Scrum emphasizes iterative development, allowing teams to adapt to changing requirements and market demands. This agility is crucial in today's fast-paced world, where rapid innovation is key to success.
* **Enhanced Collaboration:** Scrum fosters communication and teamwork, empowering teams to self-organize and take ownership of their work. Regular communication during sprints and daily stand-up meetings ensure everyone is on the same page.
* **Improved Transparency:** Scrum provides clear visibility into project progress, risks, and roadblocks. This transparency allows for timely adjustments and prevents surprises.

**Arguments against Scrum:**

* **Oversimplification:** Critics argue that Scrum simplifies complex projects and doesn't address the nuances of specific projects. It can lead to a rigid adherence to processes without considering individual project needs.
* **Focus on Short-term Goals:** Scrum's emphasis on sprints can prioritize short-term goals over long-term vision and strategic planning. This can lead to a lack of big-picture thinking and a focus on delivering incremental features rather than creating truly impactful products.
* **Burnout and Pressure:** The high-intensity nature of Scrum can lead to team burnout and increased pressure to deliver. This can hinder creativity and innovation, ultimately impacting project quality.

**Conclusion:**

Scrum's effectiveness depends heavily on the project context, team composition, and implementation. While it offers numerous advantages, its limitations should be acknowledged. Striking a balance between agile principles and considering project-specific needs is crucial for successfully utilizing Scrum and achieving desired outcomes. 

The future of Scrum lies in its ability to evolve and adapt to the changing demands of the workplace. By acknowledging its limitations and leveraging its strengths, organizations can unlock the true potential of this agile methodology and achieve impactful results.


The provided text highlights several negative aspects of Scrum methodology:

* **Oversimplification:**  Scrum can be too simplistic for complex projects, leading to a rigid adherence to processes without considering individual project needs.  This could result in ignoring important nuances and failing to adequately address project complexities.
* **Focus on Short-term Goals:** Scrum's emphasis on sprints can lead to a prioritization of short-term goals over long-term vision and strategic planning. This can result in a lack of big-picture thinking and an emphasis on incremental features rather than creating truly impactful products.
* **Burnout and Pressure:** The high-intensity nature of Scrum can lead to team burnout and increased pressure to deliver. This can hinder creativity and innovation, ultimately impacting project quality and team morale.

In addition to these points, other potential negative aspects of Scrum include:

* **Lack of Flexibility:**  While touted as flexible, Scrum can become rigid if not implemented correctly, leading to an inability to adapt to unforeseen changes or adjust to evolving project requirements.
* **Focus on Documentation:**  Scrum can emphasize documentation and meeting rituals over actual work, creating a bureaucratic overhead that hinders productivity.
* **Lack of Individual Creativity:** The highly structured nature of Scrum can stifle individual creativity and initiative, leading to a less innovative and dynamic work environment.
* **Difficult to Implement:**  Implementing Scrum effectively requires a strong understanding of its principles and a dedicated team willing to embrace the methodology.  Poor implementation can lead to confusion, inefficiency, and frustration. 

It's important to remember that Scrum is a framework, and its effectiveness depends on how it is implemented and adapted to specific project needs.  While it can be a valuable tool for managing projects, organizations should be aware of its potential drawbacks and consider alternative approaches if necessary. 


The provided text highlights several positive aspects of Scrum methodology:

**1. Increased Flexibility:** Scrum's iterative development allows teams to adapt to changing requirements and market demands. This agility is crucial in today's fast-paced environment. 

**2. Enhanced Collaboration:** Scrum promotes communication and teamwork, empowering teams to self-organize and take ownership of their work. Regular communication during sprints and daily stand-up meetings ensure everyone is on the same page.

**3. Improved Transparency:** Scrum provides clear visibility into project progress, risks, and roadblocks. This transparency allows for timely adjustments and prevents surprises.

**4. Focus on Deliverables:** Scrum's emphasis on sprints and delivering working software at the end of each sprint promotes a focus on tangible outcomes. 

**5. Improved Quality:** By iterating and getting feedback early and often, Scrum helps teams identify and address issues quickly, leading to higher quality products.

**6. Continuous Improvement:** Scrum promotes a culture of continuous improvement through retrospectives and feedback mechanisms. 

Overall, the text emphasizes the positive aspects of Scrum's ability to promote flexibility, collaboration, transparency, and continuous improvement. 


##  Scrum Methodology: Balancing the Pros and Cons 

The provided text presents a nuanced view of Scrum methodology, highlighting both its strengths and weaknesses.  While it acknowledges Scrum's value in promoting flexibility, collaboration, and transparency, it also emphasizes potential drawbacks such as oversimplification, short-term focus, and burnout. 

**Here's a summary of the key points:**

**Strengths of Scrum:**

* **Flexibility:** Adapts to changing requirements and market demands.
* **Collaboration:** Fosters communication and teamwork, leading to self-organization and ownership.
* **Transparency:** Provides clear visibility into project progress, risks, and roadblocks.
* **Focus on Deliverables:** Emphasizes sprints and delivering working software, promoting tangible outcomes.
* **Improved Quality:** Iteration and early feedback lead to quicker issue identification and higher quality products.
* **Continuous Improvement:** Encourages a culture of ongoing improvement through retrospectives and feedback. 

**Weaknesses of Scrum:**

* **Oversimplification:** Can be too simplistic for complex projects, leading to rigid adherence to processes and neglecting project nuances.
* **Short-term Focus:**  Sprint emphasis may prioritize short-term goals over long-term vision and strategic planning.
* **Burnout and Pressure:**  High-intensity nature can lead to team burnout and pressure to deliver, potentially hindering creativity and innovation.
* **Lack of Flexibility:**  Can become rigid if not implemented correctly, hindering adaptability to unforeseen changes.
* **Focus on Documentation:** Emphasis on documentation and meetings can lead to bureaucratic overhead and hinder productivity.
* **Lack of Individual Creativity:** Highly structured nature can stifle individual creativity and initiative.
* **Difficult to Implement:** Requires strong understanding of its principles and a dedicated team, poor implementation can lead to inefficiency and frustration. 

**Conclusion:**

Scrum's effectiveness is contingent on the project context, team composition, and implementation.  While its advantages are significant, organizations should be aware of its potential drawbacks.  Successfully utilizing Scrum requires:

* **Careful adaptation:**  Tailoring the framework to specific project needs and avoiding rigid adherence to processes.
* **Balancing short-term and long-term goals:**  Maintaining a focus on strategic vision alongside sprint-based work.
* **Promoting team well-being:**  Addressing potential burnout through work-life balance and fostering a supportive environment.
* **Strong leadership and implementation:** Ensuring a thorough understanding of Scrum principles and dedicated team commitment.

By acknowledging both the strengths and limitations of Scrum and implementing it thoughtfully, organizations can maximize its benefits and mitigate its risks, unlocking the true potential of this agile methodology. 


In [19]:
for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('======')


scrum methodology

## The Agile Debate: Scrum Methodology - Friend or Foe?

Scrum, a popular agile methodology, has become a mainstay in software development and beyond. Its proponents tout its flexibility, adaptability, and ability to foster collaboration, leading to increased productivity and customer satisfaction.  However, critics argue that Scrum's emphasis on short sprints, daily stand-ups, and continuous feedback can be counterproductive, leading to burnout, inefficient planning, and a lack of focus on long-term goals. 

**Arguments for Scrum:**

* **Increased Efficiency and Productivity:** Scrum's iterative nature allows for rapid feedback loops, enabling teams to identify and address problems early. This leads to faster development cycles and potentially more efficient use of resources.
* **Improved Communication and Collaboration:** Daily stand-ups, sprint reviews, and retrospectives encourage open communication and collaboration among team members, fostering a sense of shared ownership and responsibility.
* **Adaptability and Flexibility:** Scrum's iterative nature allows for flexibility in responding to changing requirements and market conditions. It enables teams to pivot quickly and adjust their approach as needed.
* **Enhanced Customer Satisfaction:** Regular feedback loops and demonstrable progress through sprint reviews lead to increased customer satisfaction and better alignment with their needs.

**Arguments Against Scrum:**

* **Burnout and Stress:** The constant pressure to deliver in short sprints can lead to burnout and increased stress levels for team members, especially in high-pressure environments.
* **Lack of Focus on Long-Term Goals:** The emphasis on short-term sprints can lead to a neglect of long-term strategic planning and vision.
* **Inefficient Planning:** The reliance on short sprints can make it difficult to plan for complex projects with long timelines and intricate dependencies.
* **Overemphasis on Rituals:** Some argue that Scrum can become overly bureaucratic, with its emphasis on daily stand-ups and sprint reviews often feeling like unnecessary rituals.

**Conclusion:**

Scrum, like any methodology, has its strengths and weaknesses. Whether it is a valuable tool for a team depends on the specific context, project complexity, and team dynamics. While its ability to foster collaboration, adapt to change, and deliver quickly is undeniable, potential drawbacks such as burnout and a lack of strategic focus must be carefully considered. 

 Ultimately, the success of Scrum lies in its effective implementation and adaptation to specific circumstances. It's crucial for teams to be mindful of the potential pitfalls and utilize Scrum's benefits strategically to achieve optimal results. 


The provided text highlights several negative aspects of Scrum methodology, including:

* **Burnout and Stress:** The constant pressure to deliver in short sprints can lead to burnout and increased stress levels for team members, especially in high-pressure environments.
* **Lack of Focus on Long-Term Goals:** The emphasis on short-term sprints can lead to a neglect of long-term strategic planning and vision.
* **Inefficient Planning:** The reliance on short sprints can make it difficult to plan for complex projects with long timelines and intricate dependencies.
* **Overemphasis on Rituals:** Some argue that Scrum can become overly bureaucratic, with its emphasis on daily stand-ups and sprint reviews often feeling like unnecessary rituals.

These negative aspects can be summarized as:

* **Potential for employee well-being issues:** Scrum's intense pace and focus on short-term deliverables can lead to burnout and stress.
* **Limited focus on long-term vision:** The emphasis on sprints may hinder the development of long-term strategies and plans.
* **Challenges in planning complex projects:** Short sprints may not be suitable for large, complex projects with intricate dependencies.
* **Risk of excessive bureaucracy:** The rituals and ceremonies associated with Scrum can become burdensome and unproductive.

It is important to note that the effectiveness of Scrum depends heavily on its implementation and adaptation to specific contexts. While it offers benefits like increased efficiency and collaboration, its drawbacks should be carefully considered to avoid potential pitfalls and ensure its successful application. 


The provided text highlights several positive aspects of the Scrum methodology. Let's break them down:

**Increased Efficiency and Productivity:**

* **Iterative nature:** This allows for rapid feedback loops, enabling teams to identify and address problems early, leading to faster development cycles.
* **Resource optimization:** By focusing on delivering working software in short sprints, teams can optimize resource utilization and avoid wasted effort on non-essential tasks.

**Improved Communication and Collaboration:**

* **Daily stand-ups:** Promote open and frequent communication, ensuring everyone is aligned on progress and potential roadblocks.
* **Sprint reviews and retrospectives:** Provide opportunities for collective reflection, allowing the team to learn from their experiences and make adjustments for future sprints.

**Adaptability and Flexibility:**

* **Iterative approach:** Allows for flexibility in responding to changing requirements and market conditions, enabling teams to pivot quickly and adjust their approach as needed.
* **Embracing change:** Scrum emphasizes the importance of adapting to changing circumstances, making it suitable for projects with evolving needs and requirements.

**Enhanced Customer Satisfaction:**

* **Regular feedback loops:** Ensure customers are involved in the development process, providing valuable input and ensuring the final product meets their expectations.
* **Demonstrable progress:** Sprint reviews allow customers to see the progress made and gain confidence in the team's ability to deliver. 

Overall, the text acknowledges the potential benefits of Scrum in fostering a collaborative, adaptable, and efficient work environment. It's important to note that the success of Scrum relies on its effective implementation and adaptation to specific contexts. 


## Scrum Methodology: Balancing Benefits and Challenges

Scrum, a popular agile methodology, presents a compelling case for its potential to boost efficiency, collaboration, and customer satisfaction. However, it also comes with its own set of drawbacks,  including the potential for employee burnout, lack of focus on long-term vision, difficulties in planning complex projects, and a risk of becoming overly bureaucratic. 

**The Pros:**

* **Increased Efficiency and Productivity:** Scrum's iterative nature facilitates rapid feedback loops and resource optimization, leading to faster development cycles.
* **Improved Communication and Collaboration:** Daily stand-ups, sprint reviews, and retrospectives foster open communication, shared responsibility, and a collaborative work environment.
* **Adaptability and Flexibility:** Scrum's iterative approach allows teams to adapt to changing requirements and market conditions, ensuring they can pivot and adjust as needed.
* **Enhanced Customer Satisfaction:** Regular feedback loops and demonstrable progress through sprint reviews build trust and ensure alignment with customer needs.

**The Cons:**

* **Potential for Employee Well-being Issues:** The constant pressure to deliver in short sprints can lead to burnout and stress, particularly in high-pressure environments.
* **Limited Focus on Long-Term Vision:** The emphasis on sprints may hinder the development of long-term strategies and plans.
* **Challenges in Planning Complex Projects:** Short sprints may not be suitable for large, complex projects with intricate dependencies.
* **Risk of Excessive Bureaucracy:** The rituals and ceremonies associated with Scrum can become burdensome and unproductive.

**The Verdict:**

The success of Scrum lies in its effective implementation and adaptation to specific contexts. While it offers undeniable benefits, its potential drawbacks must be carefully considered to ensure its successful application. Teams should strive to balance the benefits of increased efficiency and collaboration with the need to maintain employee well-being, focus on long-term goals, and navigate complex projects strategically.  By implementing Scrum thoughtfully and adapting it to specific circumstances, teams can leverage its benefits while minimizing the potential pitfalls. 
